In [ ]:
import pandas as pd
import ast 

acc_df = pd.read_csv('고속도로 사고 데이터.csv')
acc_df

In [ ]:
traffic_df = pd.read_csv('도로종류별 교통량(좌표,휴게소,사고건수,화물차휴게소)추가_최종본.csv')

In [ ]:
traffic_df.head()
traffic_df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
trafffic_ndf = traffic_df[traffic_df['accident_count'] > 0]
trafffic_ndf.head()

In [ ]:
traffic_df['startpoint_coor'] = traffic_df['startpoint_coor'].apply(ast.literal_eval)
traffic_df['endpoint_coor'] = traffic_df['endpoint_coor'].apply(ast.literal_eval)

In [ ]:
x = []
y = []
for i in range(len(traffic_df)):
    start_x, start_y = traffic_df['startpoint_coor'][i]
    end_x, end_y = traffic_df['endpoint_coor'][i]
    
    center_x = (start_x + end_x) / 2
    center_y = (start_y + end_y) / 2
    
    x.append(center_x)
    y.append(center_y)

traffic_df['center_x'] = x
traffic_df['center_y'] = y

In [ ]:
coord_ec = traffic_df[['center_x','center_y']].astype(float).values.tolist()
cs = traffic_df['구간'].tolist()
cc = traffic_df['accident_count'].tolist()

In [ ]:
# 라이브러리
import folium
from folium import Choropleth, Circle, Marker, PolyLine
import numpy as np
import pandas as pd
import seaborn as sns
import squarify
import ast
import pickle
import warnings
import math

import geopy
from geopy.distance import geodesic

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

from mpl_toolkits.mplot3d import Axes3D 
from datetime import datetime
from haversine import haversine

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, QuantileTransformer, RobustScaler

In [ ]:
hwamulcha_df = acc_df[acc_df['피해운전자 차종'] == '화물']
hwamulcha_df.head()

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium

map = folium.Map(location=[36.5,128], tiles='cartodbpositron', 
                        zoom_start=7.5)
#고속도로 시각화
for i in range(len(traffic_df)):
    start_x_coor, start_y_coor = traffic_df['startpoint_coor'][i]
    end_x_coor, end_y_coor = traffic_df['endpoint_coor'][i]
    polylines = []
    polylines.append([(start_x_coor,start_y_coor),(end_x_coor,end_y_coor)])

    folium.PolyLine(locations=polylines,
                 color="#B7D7B7",
                 weight = 5,
                 fill_color = None,
                 fill_opacity=0.2,
                 tooltip=traffic_df['구간'][i]
                ).add_to(map)


#고속도로 사고 위도,경도
for name, lat, lng in zip(hwamulcha_df.시군구, hwamulcha_df.위도, hwamulcha_df.경도):
    folium.CircleMarker([lat, lng],
                        radius=6,         # 원의 반지름
                        color='brown',         # 원의 둘레 색상
                        weight=0.1,
                        fill=True,
                        fill_color='#ff0000',    # 원을 채우는 색
                        fill_opacity=0.5, # 투명도    
                        popup=name
    ).add_to(map)
    
display(map)

In [ ]:
traffic_df.info()

In [ ]:
traffic_df['startpoint_coor'] = traffic_df['startpoint_coor'].apply(ast.literal_eval)
traffic_df['endpoint_coor'] = traffic_df['endpoint_coor'].apply(ast.literal_eval)

In [ ]:
def cal_rad(A,B):
    rad = math.atan2(A[1]-B[1], A[0]-B[0])
    return rad

def count_color(count):
    if count >=20:
        color = '#c18cd1'
    elif count >=14:
        color = '#bc65d6'
    elif count >=7:
        color = '#b940de'
    elif count >=3:
        color = '#b223de'
    else:
        color = '#9000c9'
    return color

In [ ]:
traffic_df.head()

In [ ]:
counts=[]
acc_map = folium.Map(location=[36, 127.5], tiles='cartodbpositron', zoom_start=7)

for i in range(len(traffic_df)):
    startpoint = traffic_df['startpoint_coor'][i]
    endpoint = traffic_df['endpoint_coor'][i]

    rad = cal_rad(startpoint, endpoint)
    deg = np.rad2deg(rad)
    dist = 5
    origin_start = geopy.Point(startpoint)
    origin_end = geopy.Point(endpoint)

    degree = [90+deg, 270+deg]

    poly_a = []
    poly_b = []

    for de in degree:

        destination_start = geodesic(kilometers=dist).destination(origin_start, de)
        destination_end = geodesic(kilometers=dist).destination(origin_end, de)
        new_lat1, new_lon1 = destination_start.latitude, destination_start.longitude
        new_lat2, new_lon2 = destination_end.latitude, destination_end.longitude

        poly_a.append([new_lat1, new_lon1])
        poly_b.append([new_lat2, new_lon2])
        
      
    polygon_vect = np.array([poly_a[0],poly_b[0],poly_b[1],poly_a[1]])
    polygon = Polygon(polygon_vect)

    count =0
    for i in range(len(acc_df)):
        point = Point(acc_df['위도'][i], acc_df['경도'][i])

        if polygon.contains(point):
            count+=1

    counts.append(count)
    
    lats_vect = np.array([poly_a[0], poly_b[0]])
    lons_vect = np.array([poly_a[1], poly_b[1]])
    folium.Polygon(locations=[poly_a[0], poly_b[0], poly_b[1], poly_a[1]],
                   weight = 1,
                   color = None, 
                   fill_color = '#50aa50',
                   fill_opacity=0.2, #0.2
                      ).add_to(acc_map)
    
traffic_df['counts'] = counts
    
for i,co in enumerate(coord_ec):
    folium.Circle(location=co,color="red", opacity = 0.2 * cc[i], radius=cc[i] * 70,fill_color=f'#C1C0BD' ,fill_opacity=0.5).add_to(acc_map)

In [ ]:
acc_map

In [ ]:
acc_map.save('고속도로 사고 시각화_최종본.html')